<a href="https://colab.research.google.com/github/lucasnbd/1TSCG-DbSuporte/blob/main/ex_RegLin_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import statsmodels.api as sm


In [ ]:
# carregando dataset
dados = pd.read_csv("pns-novo.csv", sep="\t")
dados.head(5)

FileNotFoundError: [Errno 2] No such file or directory: 'pns-novo.csv'

In [ ]:
# removendo colunas que não sõa interessantes para a analise
dados = dados.drop(columns = ['Estado', 'Escolaridade', 'Qdo teve ultima consulta odontologica?', 'Nas ultimas 2 semanas, pq nao foi ao posto de saude?'])
dados

In [ ]:
# contando quantos registros NaN temos no dataset
dados.isna().sum()

In [ ]:
# removendo registros NaN
dados = dados.dropna()
dados

In [ ]:
# dados para calculo
dados['Situacao censitaria'].unique()

In [ ]:
dados['Problemas de saude nas ultimas Nao semanas'].unique()

In [ ]:
dados['Qdo teve ultima consulta medica?'].unique()

In [ ]:
dados['Como voce avalia a sua saude?'].unique()

In [ ]:
# criando colunas para calculos

dados["Urbano"] = dados.apply(lambda linha: 1 if linha["Situacao censitaria"] == "Urbano" else 0, axis=1)
dados["Rural"] = dados.apply(lambda linha: 1 if linha["Situacao censitaria"] == "Rural" else 0, axis=1)
dados["Sem probl saude"] = dados.apply(lambda linha: 1 if linha["Problemas de saude nas ultimas Nao semanas"] == "Nao" else 0, axis=1)
dados["Com probl saude"] = dados.apply(lambda linha: 1 if linha["Problemas de saude nas ultimas Nao semanas"] == "Sim" else 0, axis=1)
dados["<= 1"] = dados.apply(lambda linha: 1 if linha["Qdo teve ultima consulta medica?"] == "Ate 1 ano" else 0, axis=1)
dados["> 1 <= 2"] = dados.apply(lambda linha: 1 if linha["Qdo teve ultima consulta medica?"] == "Mais de 1 ano a 2 anos" else 0, axis=1)
dados["> 2 <= 3"] = dados.apply(lambda linha: 1 if linha["Qdo teve ultima consulta medica?"] == "Mais de 2 anos a 3 anos" else 0, axis=1)
dados["> 3"] = dados.apply(lambda linha: 1 if linha["Qdo teve ultima consulta medica?"] == "Mais de 3 anos" else 0, axis=1)
dados["Nunca_foi_medico"] = dados.apply(lambda linha: 1 if linha["Qdo teve ultima consulta medica?"] == "Nunca foi ao medico" else 0, axis=1)
dados["Muito ruim"] = dados.apply(lambda linha: 1 if linha["Como voce avalia a sua saude?"] == "Muito ruim" else 0, axis=1)
dados["Ruim"] = dados.apply(lambda linha: 1 if linha["Como voce avalia a sua saude?"] == "Ruim" else 0, axis=1)
dados["Regular"] = dados.apply(lambda linha: 1 if linha["Como voce avalia a sua saude?"] == "Regular" else 0, axis=1)
dados["Boa"] = dados.apply(lambda linha: 1 if linha["Como voce avalia a sua saude?"] == "Boa" else 0, axis=1)
dados["Muito boa"] = dados.apply(lambda linha: 1 if linha["Como voce avalia a sua saude?"] == "Muito boa" else 0, axis=1)
dados

In [ ]:
# arrumando dados para IMC
dados['Altura'] = dados["Altura"] / 100
dados["IMC"] = round(dados['Peso'] / (dados['Altura'] ** 2),2)
dados

In [ ]:
# normalizando a coluna IMC

imc_max = dados["IMC"].max()
imc_min = dados["IMC"].min()

dados["IMC_Normalizado"] = (dados['IMC'] - imc_min) / (imc_max - imc_min)

In [ ]:
  dados.columns

In [ ]:
# começando modelo de RLM

modelo = LinearRegression(n_jobs=None, fit_intercept=True, copy_X=True)

x = dados[[ 'Muito ruim', 'Ruim', 'Regular', 'Boa', 'Muito boa','Sem probl saude', '<= 1', '> 1 <= 2', '> 2 <= 3', '> 3']].to_numpy()
y = dados[['IMC']]
modelo.fit(x, y)
modelo.score(x, y)

In [ ]:
x = sm.add_constant(x)
modelo = sm.OLS(y, x).fit()
print(modelo.summary())
